In [1]:
import torch
from torch.nn import functional as F
from torch import nn
from torch.optim import Adam
from torch.utils.data import DataLoader, random_split
from keras.datasets import mnist
import os
from torchvision import datasets, transforms

Using TensorFlow backend.


In [2]:
class AutoEncoder(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.encoder = nn.Sequential(nn.Linear(28*28, 64), nn.ReLU(), nn.Linear(64, 3))
        self.decoder = nn.Sequential(nn.Linear(3, 64), nn.ReLU(), nn.Linear(64, 28*28))
        
    def forward(self, x):
        z = self.encoder(x)
        x = self.decoder(z)
        return z, x

model = AutoEncoder().cuda(0)

In [3]:
(mnist_train, _), _ = mnist.load_data()

mnist_train = torch.Tensor(mnist_train)
mnist_train /= 255.

mnist_train, mnist_val = random_split(mnist_train, [55000, 5000])

mnist_train = DataLoader(mnist_train, batch_size=64)
mnist_val = DataLoader(mnist_val, batch_size=64)

In [4]:
optimizer = Adam(model.parameters(), lr=1e-3)

In [5]:
model.train()
for epoch in range(1):
    total_loss = 0
    for train_batch in mnist_train:
        optimizer.zero_grad()
        x = train_batch.cuda(0)
        x = x.view(x.size(0), -1)
        z, x_hat = model(x)
        loss = F.mse_loss(x_hat, x)
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
    print('train_loss:', total_loss/55000)

train_loss: 0.0007822969600558281


In [6]:
model.eval()
with torch.no_grad():
    val_loss = 0
    for val_batch in mnist_val:
        x = val_batch.cuda(0)
        x = x.view(x.size(0), -1)
        z, x_hat = model(x)
        loss = F.mse_loss(x_hat, x)
        val_loss += loss.item()
    print('val_loss:', val_loss/5000)

val_loss: 0.0006770172156393528
